In [5]:
import pandas as pd
import numpy as np
import NB_Funcs as hlp
from scipy.ndimage import convolve
from matplotlib import pyplot as plt
import pickle

# Initial Population Model

$M(x) = R(0) + \nu*N(0, x)$

$dN/dt = r*(M - \nu*N)*N$

$dR/dt = -\nu*dN/dt$

$r = 0.5, \nu = 2*10^{-7}$

In [6]:
N = np.load('../data/RawData/VIN/VIN_N_curves.npy')

N = N[:,:250,:,:]

N_stnd = np.array(N, copy = True)
dt=0.1
time = np.arange(N.shape[1])*dt
plates = N.shape[0]
t = N.shape[1]
rows = N.shape[2]
cols = N.shape[3]

plate_maximums = {}

for p in range(plates):
    plate_maximums[p] = np.max(N[p,:,:,:])
    N_stnd[p,:,:,:] = N_stnd[p,:,:,:] / np.max(N[p,:,:,:])
    
print(N.shape)
print(N_stnd.shape)

(4, 250, 32, 48)
(4, 250, 32, 48)


In [8]:
# 32x48 array of ring value
rings = np.empty((rows,cols))

edge_r = (0,rows-1)
edge_c = (0,cols-1)

for i in range(rows):
    for j in range(cols):
        
        rings[i,j] = min(abs(i-edge_r[0])+1,abs(i-edge_r[1])+1,
                             abs(j-edge_c[0])+1,abs(j-edge_c[1])+1)

# Preprocessing

In [9]:
#2 different ways of storing the data, ring format and colony format

ring_form_data = {}

colony_form_data = {}

## Colony Format

In [10]:
for p in range(plates):
    colony_form_data[p] = []
    
    for i in range(rows):
        for j in range(cols):
            Ps = np.ones((t)) * p
            Is = np.ones((t)) * i
            Js = np.ones((t)) * j
            N_init = np.ones((t)) * N[p,0,i,j]
            N_ = N[p,:,i,j]
            N_init_stnd = np.ones((t)) * N_stnd[p,0,i,j]
            N_stnd_ = N_stnd[p,:,i,j]
            
            
            data = np.vstack([time,Ps,Is,Js,N_init,N_,N_init_stnd,N_stnd_]).T
            df = pd.DataFrame(data = data, columns = ["time","plate","i","j","N_init","Pop",
                                                      "N_init_stnd","Pop_stnd"])
            if j == 0:
                colony_form_data[p].append([df])
            else: colony_form_data[p][i].append(df)
            
for p in range(plates):
    dNdt = hlp.cubic_splines(N[p],time)
    dNdt_stnd = hlp.cubic_splines(N_stnd[p],time)
    for i in range(rows):
        for j in range(cols):
            colony_form_data[p][i][j]['dNdt'] = dNdt[:,i,j]
            colony_form_data[p][i][j]['dNdt_stnd'] = dNdt_stnd[:,i,j]
            
for p in range(plates):
    Nbar = np.empty((t,rows,cols))

    kernel = np.array([[1,1,1],
                       [1,0,1],
                       [1,1,1]])
    
    
    for tic in range(t):

        sums = convolve(N[p,tic,:,:],kernel,mode='constant')
        
        Nbar[tic,:,:] = sums / 8 #we deem every colony to have 8 neighbors to reflect differences in averages
                                    #between border colonies and others

    for i in range(rows):
        for j in range(cols):
            colony_form_data[p][i][j]['Nbar'] = Nbar[:,i,j]
            colony_form_data[p][i][j]['Cum_Nbar'] = np.cumsum(Nbar[:,i,j])*dt
            colony_form_data[p][i][j]['Cum_N'] = np.cumsum(N[p,:,i,j])*dt
            
            colony_form_data[p][i][j]['Nbar_stnd'] = Nbar[:,i,j] / np.max(N)
            colony_form_data[p][i][j]['Cum_Nbar_stnd'] = np.cumsum(Nbar[:,i,j])*dt / np.max(N)
            colony_form_data[p][i][j]['Cum_N_stnd'] = np.cumsum(N[p,:,i,j])*dt / np.max(N)
            
            

for p in range(plates):
    for i in range(rows):
        for df in colony_form_data[p][i]:
            df['dNovN'] = df['dNdt'] / df['Pop']
            df['dNovN_stnd'] = df['dNdt_stnd'] / df['Pop_stnd']

for p in range(plates):
    for i in range(rows):
        for j in range(cols):
            colony_form_data[p][i][j]['Ring'] = np.ones((t)) * rings[i,j]
    
colony_form_data[0][0][0].head(3)

,time,plate,i,j,N_init,Pop,N_init_stnd,Pop_stnd,dNdt,dNdt_stnd,Nbar,Cum_Nbar,Cum_N,Nbar_stnd,Cum_Nbar_stnd,Cum_N_stnd,dNovN,dNovN_stnd,Ring
0,0.0,0.0,0.0,0.0,122705.342189,122705.342189,0.022864,0.022864,59903.823051,0.011162,45512.986141,4551.298614,12270.534219,0.008183,0.000818,0.002206,0.488192,0.488192,1.0
1,0.1,0.0,0.0,0.0,122705.342189,128840.609299,0.022864,0.024007,62824.241717,0.011706,47788.635448,9330.162159,25154.595149,0.008592,0.001677,0.004523,0.487612,0.487612,1.0
2,0.2,0.0,0.0,0.0,122705.342189,135274.735048,0.022864,0.025206,65880.995850,0.012276,50175.159484,14347.678107,38682.068654,0.009021,0.002580,0.006955,0.487016,0.487016,1.0


## Ring Format

In [11]:
for p in range(plates):
    checklist = {}
    for i in range(rows):
        for df in colony_form_data[p][i]:
            ring_val = df['Ring'][0]
            if ring_val in checklist:
                checklist[ring_val].append(df)
            else:
                checklist[ring_val] = [df]
    
    ring_form_data[p] = checklist
    
print(len(ring_form_data[0][16]))
ring_form_data[0][1][0].head(3)

36


,time,plate,i,j,N_init,Pop,N_init_stnd,Pop_stnd,dNdt,dNdt_stnd,Nbar,Cum_Nbar,Cum_N,Nbar_stnd,Cum_Nbar_stnd,Cum_N_stnd,dNovN,dNovN_stnd,Ring
0,0.0,0.0,0.0,0.0,122705.342189,122705.342189,0.022864,0.022864,59903.823051,0.011162,45512.986141,4551.298614,12270.534219,0.008183,0.000818,0.002206,0.488192,0.488192,1.0
1,0.1,0.0,0.0,0.0,122705.342189,128840.609299,0.022864,0.024007,62824.241717,0.011706,47788.635448,9330.162159,25154.595149,0.008592,0.001677,0.004523,0.487612,0.487612,1.0
2,0.2,0.0,0.0,0.0,122705.342189,135274.735048,0.022864,0.025206,65880.995850,0.012276,50175.159484,14347.678107,38682.068654,0.009021,0.002580,0.006955,0.487016,0.487016,1.0


## Functions for Storing and Loading Results

In [12]:
def pickle_save(filename, data):
    with open(filename,'wb') as _save:
        pickle.dump(data,_save)
    
def pickle_load(filename):
    with open(filename,'rb') as _load:
        var = pickle.load(_load)
    return var

In [13]:
pickle_save('../Data/Train_Test/VIN_PreprocessedData',(colony_form_data,ring_form_data,plate_maximums))